In [3]:
import numpy as np
import pandas as pd
import gc
import time
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
def get_dataset():
    app_train = pd.read_csv('data/application_train.csv')
    app_test = pd.read_csv('data/application_test.csv')
    apps = pd.concat([app_train, app_test])
    prev = pd.read_csv('data/previous_application.csv')

    return apps, prev

apps, prev = get_dataset()
    

In [6]:
def get_apps_processed(apps):
    
    # EXT_SOURCE_X FEATURE 가공
    apps['APPS_EXT_SOURCE_MEAN'] = apps[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
    apps['APPS_EXT_SOURCE_STD'] = apps[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].std(axis=1)
    apps['APPS_EXT_SOURCE_STD'] = apps['APPS_EXT_SOURCE_STD'].fillna(apps['APPS_EXT_SOURCE_STD'].mean())
    
    # AMT_CREDIT 비율로 Feature 가공
    apps['APPS_ANNUITY_CREDIT_RATIO'] = apps['AMT_ANNUITY']/apps['AMT_CREDIT']
    apps['APPS_GOODS_CREDIT_RATIO'] = apps['AMT_GOODS_PRICE']/apps['AMT_CREDIT']
    
    # AMT_INCOME_TOTAL 비율로 Feature 가공
    apps['APPS_ANNUITY_INCOME_RATIO'] = apps['AMT_ANNUITY']/apps['AMT_INCOME_TOTAL']
    apps['APPS_CREDIT_INCOME_RATIO'] = apps['AMT_CREDIT']/apps['AMT_INCOME_TOTAL']
    apps['APPS_GOODS_INCOME_RATIO'] = apps['AMT_GOODS_PRICE']/apps['AMT_INCOME_TOTAL']
    apps['APPS_CNT_FAM_INCOME_RATIO'] = apps['AMT_INCOME_TOTAL']/apps['CNT_FAM_MEMBERS']
    
    # DAYS_BIRTH, DAYS_EMPLOYED 비율로 Feature 가공
    apps['APPS_EMPLOYED_BIRTH_RATIO'] = apps['DAYS_EMPLOYED']/apps['DAYS_BIRTH']
    apps['APPS_INCOME_EMPLOYED_RATIO'] = apps['AMT_INCOME_TOTAL']/apps['DAYS_EMPLOYED']
    apps['APPS_INCOME_BIRTH_RATIO'] = apps['AMT_INCOME_TOTAL']/apps['DAYS_BIRTH']
    apps['APPS_CAR_BIRTH_RATIO'] = apps['OWN_CAR_AGE'] / apps['DAYS_BIRTH']
    apps['APPS_CAR_EMPLOYED_RATIO'] = apps['OWN_CAR_AGE'] / apps['DAYS_EMPLOYED']
    
    return apps

In [7]:
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier

def get_prev_processed(prev):
    # 대출 신청 금액과 실제 대출액/대출 상품금액 차이 및 비율
    prev['PREV_CREDIT_DIFF'] = prev['AMT_APPLICATION'] - prev['AMT_CREDIT']
    prev['PREV_GOODS_DIFF'] = prev['AMT_APPLICATION'] - prev['AMT_GOODS_PRICE']
    prev['PREV_CREDIT_APPL_RATIO'] = prev['AMT_CREDIT']/prev['AMT_APPLICATION']
    # prev['PREV_ANNUITY_APPL_RATIO'] = prev['AMT_ANNUITY']/prev['AMT_APPLICATION']
    prev['PREV_GOODS_APPL_RATIO'] = prev['AMT_GOODS_PRICE']/prev['AMT_APPLICATION']
    
    prev['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace= True)
    prev['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace= True)
    prev['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace= True)
    prev['DAYS_LAST_DUE'].replace(365243, np.nan, inplace= True)
    prev['DAYS_TERMINATION'].replace(365243, np.nan, inplace= True)
    # 첫번째 만기일과 마지막 만기일까지의 기간
    prev['PREV_DAYS_LAST_DUE_DIFF'] = prev['DAYS_LAST_DUE_1ST_VERSION'] - prev['DAYS_LAST_DUE']
    # 매월 납부 금액과 납부 횟수 곱해서 전체 납부 금액 구함. 
    all_pay = prev['AMT_ANNUITY'] * prev['CNT_PAYMENT']
    # 전체 납부 금액 대비 AMT_CREDIT 비율을 구하고 여기에 다시 납부횟수로 나누어서 이자율 계산. 
    prev['PREV_INTERESTS_RATE'] = (all_pay/prev['AMT_CREDIT'] - 1)/prev['CNT_PAYMENT']
        
    return prev
    
    
def get_prev_amt_agg(prev):
    # 새롭게 생성된 대출 신청액 대비 다른 금액 차이 및 비율로 aggregation 수행. 
    agg_dict = {
         # 기존 컬럼. 
        'SK_ID_CURR':['count'],
        'AMT_CREDIT':['mean', 'max', 'sum'],
        'AMT_ANNUITY':['mean', 'max', 'sum'], 
        'AMT_APPLICATION':['mean', 'max', 'sum'],
        'AMT_DOWN_PAYMENT':['mean', 'max', 'sum'],
        'AMT_GOODS_PRICE':['mean', 'max', 'sum'],
        'RATE_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'DAYS_DECISION': ['min', 'max', 'mean'],
        'CNT_PAYMENT': ['mean', 'sum'],
        # 가공 컬럼
        'PREV_CREDIT_DIFF':['mean', 'max', 'sum'], 
        'PREV_CREDIT_APPL_RATIO':['mean', 'max'],
        'PREV_GOODS_DIFF':['mean', 'max', 'sum'],
        'PREV_GOODS_APPL_RATIO':['mean', 'max'],
        'PREV_DAYS_LAST_DUE_DIFF':['mean', 'max', 'sum'],
        'PREV_INTERESTS_RATE':['mean', 'max']
    }

    prev_group = prev.groupby('SK_ID_CURR')
    prev_amt_agg = prev_group.agg(agg_dict)

    # multi index 컬럼을 '_'로 연결하여 컬럼명 변경
    prev_amt_agg.columns = ["PREV_"+ "_".join(x).upper() for x in prev_amt_agg.columns.ravel()]
    
    return prev_amt_agg

def get_prev_refused_appr_agg(prev):
    # 원래 groupby 컬럼 + 세부 기준 컬럼으로 groupby 수행. 세분화된 레벨로 aggregation 수행 한 뒤에 unstack()으로 컬럼레벨로 변형. 
    prev_refused_appr_group = prev[prev['NAME_CONTRACT_STATUS'].isin(['Approved', 'Refused'])].groupby([ 'SK_ID_CURR', 'NAME_CONTRACT_STATUS'])
    prev_refused_appr_agg = prev_refused_appr_group['SK_ID_CURR'].count().unstack()
    # 컬럼명 변경. 
    prev_refused_appr_agg.columns = ['PREV_APPROVED_COUNT', 'PREV_REFUSED_COUNT' ]
    # NaN값은 모두 0으로 변경. 
    prev_refused_appr_agg = prev_refused_appr_agg.fillna(0)
    
    return prev_refused_appr_agg

    

def get_prev_agg(prev):
    prev = get_prev_processed(prev)
    prev_amt_agg = get_prev_amt_agg(prev)
    prev_refused_appr_agg = get_prev_refused_appr_agg(prev)
    
    # prev_amt_agg와 조인. 
    prev_agg = prev_amt_agg.merge(prev_refused_appr_agg, on='SK_ID_CURR', how='left')
    # SK_ID_CURR별 과거 대출건수 대비 APPROVED_COUNT 및 REFUSED_COUNT 비율 생성. 
    prev_agg['PREV_REFUSED_RATIO'] = prev_agg['PREV_REFUSED_COUNT']/prev_agg['PREV_SK_ID_CURR_COUNT']
    prev_agg['PREV_APPROVED_RATIO'] = prev_agg['PREV_APPROVED_COUNT']/prev_agg['PREV_SK_ID_CURR_COUNT']
    # 'PREV_REFUSED_COUNT', 'PREV_APPROVED_COUNT' 컬럼 drop 
    prev_agg = prev_agg.drop(['PREV_REFUSED_COUNT', 'PREV_APPROVED_COUNT'], axis=1)
    
    return prev_agg

def get_apps_all_with_prev_agg(apps, prev):
    apps_all =  get_apps_processed(apps)
    prev_agg = get_prev_agg(prev)
    print('prev_agg shape:', prev_agg.shape)
    print('apps_all before merge shape:', apps_all.shape)
    apps_all = apps_all.merge(prev_agg, on='SK_ID_CURR', how='left')
    print('apps_all after merge with prev_agg shape:', apps_all.shape)
    
    return apps_all

def get_apps_all_encoded(apps_all):
    object_columns = apps_all.dtypes[apps_all.dtypes == 'object'].index.tolist()
    for column in object_columns:
        apps_all[column] = pd.factorize(apps_all[column])[0]
    
    return apps_all

def get_apps_all_train_test(apps_all):
    apps_all_train = apps_all[~apps_all['TARGET'].isnull()]
    apps_all_test = apps_all[apps_all['TARGET'].isnull()]

    apps_all_test = apps_all_test.drop('TARGET', axis=1)
    
    return apps_all_train, apps_all_test
    
def train_apps_all(apps_all_train):
    ftr_app = apps_all_train.drop(['SK_ID_CURR', 'TARGET'], axis=1)
    target_app = apps_all_train['TARGET']

    train_x, valid_x, train_y, valid_y = train_test_split(ftr_app, target_app, test_size=0.3, random_state=2020)
    print('train shape:', train_x.shape, 'valid shape:', valid_x.shape)
    clf = LGBMClassifier(
                nthread=4,
                n_estimators=2000,
                learning_rate=0.01,
                num_leaves=32,
                colsample_bytree=0.8,
                subsample=0.8,
                max_depth=8,
                reg_alpha=0.04,
                reg_lambda=0.07,
                min_child_weight=40,
                silent=-1,
                verbose=-1,
                )

    clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], eval_metric= 'auc', verbose= 100, 
                early_stopping_rounds= 100)
    
    return clf

In [8]:
apps_all = get_apps_all_with_prev_agg(apps, prev)
apps_all = get_apps_all_encoded(apps_all)
apps_all_train, apps_all_test = get_apps_all_train_test(apps_all)
ftr_app = apps_all_train.drop(['SK_ID_CURR', 'TARGET'], axis=1)
target_app = apps_all_train['TARGET']
train_x, valid_x, train_y, valid_y = train_test_split(ftr_app, target_app, test_size=0.3, random_state=2020)


/var/folders/7d/g6cftgkn73xdpn_wddqnr2y80000gn/T/ipykernel_9737/1480129953.py:53: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  prev_amt_agg.columns = ["PREV_"+ "_".join(x).upper() for x in prev_amt_agg.columns.ravel()]


prev_agg shape: (338857, 41)
apps_all before merge shape: (356255, 135)
apps_all after merge with prev_agg shape: (356255, 176)


In [9]:
from bayes_opt import BayesianOptimization
from sklearn.metrics import roc_auc_score
from lightgbm import LGBMClassifier

In [10]:
bayesian_params = {
  'max_depth': (6, 16),
  'num_leaves': (24, 64),
  'min_child_samples': (10, 200),
  'min_child_weight': (1, 50),
  'subsample': (0.5, 1),
  'colsample_bytree': (0.5, 1),
  'max_bin': (10, 500),
  'reg_lambda': (0.001, 10),
  'reg_alpha': (0.01, 50)
}

In [11]:
def lgb_roc_eval(max_depth, num_leaves, min_child_samples, min_child_weight, subsample, 
                colsample_bytree, max_bin, reg_lambda, reg_alpha):
    params = {
      'n_estimators': 500,
      'learning_rate': 0.02,
      'max_depth': int(round(max_depth)),
      'num_leaves': int(round(num_leaves)),
      'min_child_samples': int(round(min_child_samples)),
      'min_child_weight': int(round(min_child_weight)),
      'subsample': max(min(subsample, 1), 0),
      'colsample_bytree': max(min(colsample_bytree, 1), 0),
      'max_bin': max(int(round(max_bin)), 10),
      'reg_lambda': max(reg_lambda, 0),
      'reg_alpha': max(reg_alpha, 0),
    }

    lgb_model = LGBMClassifier(**params)
    lgb_model.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], eval_metric='auc',
                  verbose=100, early_stopping_rounds=100)
    valid_proba = lgb_model.predict_proba(valid_x)[:, 1]
    roc_auc = roc_auc_score(valid_y, valid_proba)

    return roc_auc


In [13]:
lgb_BO = BayesianOptimization(f=lgb_roc_eval, pbounds=bayesian_params, random_state=0)
lgb_BO.maximize(init_points=5, n_iter=25)

|   iter    |  target   | colsam... |  max_bin  | max_depth | min_ch... | min_ch... | num_le... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------


/usr/local/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[100]	training's auc: 0.772707	training's binary_logloss: 0.244888	valid_1's auc: 0.756927	valid_1's binary_logloss: 0.248271
[200]	training's auc: 0.792101	training's binary_logloss: 0.236721	valid_1's auc: 0.767671	valid_1's binary_logloss: 0.243653
[300]	training's auc: 0.805367	training's binary_logloss: 0.231617	valid_1's auc: 0.772657	valid_1's binary_logloss: 0.241864
[400]	training's auc: 0.816192	training's binary_logloss: 0.227558	valid_1's auc: 0.775109	valid_1's binary_logloss: 0.240987
[500]	training's auc: 0.825867	training's binary_logloss: 0.223969	valid_1's auc: 0.776778	valid_1's binary_logloss: 0.240419
|  1        |  0.7768   |  0.7744   |  360.4    |  12.03    |  113.5    |  21.76    |  49.84    |  21.88    |  8.918    |  0.9818   |


/usr/local/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[100]	training's auc: 0.763107	training's binary_logloss: 0.247151	valid_1's auc: 0.753866	valid_1's binary_logloss: 0.248954
[200]	training's auc: 0.781504	training's binary_logloss: 0.240001	valid_1's auc: 0.765904	valid_1's binary_logloss: 0.244142
[300]	training's auc: 0.792433	training's binary_logloss: 0.235886	valid_1's auc: 0.771276	valid_1's binary_logloss: 0.242175
[400]	training's auc: 0.800873	training's binary_logloss: 0.232837	valid_1's auc: 0.773878	valid_1's binary_logloss: 0.241263
[500]	training's auc: 0.807847	training's binary_logloss: 0.230329	valid_1's auc: 0.775323	valid_1's binary_logloss: 0.240774
|  2        |  0.7753   |  0.6917   |  397.9    |  11.29    |  117.9    |  46.35    |  26.84    |  4.366    |  0.2032   |  0.9163   |


...

[100]	training's auc: 0.778972	training's binary_logloss: 0.243165	valid_1's auc: 0.75843	valid_1's binary_logloss: 0.247778
[200]	training's auc: 0.80031	training's binary_logloss: 0.234009	valid_1's auc: 0.769042	valid_1's binary_logloss: 0.243133
[300]	training's auc: 0.816052	training's binary_logloss: 0.227925	valid_1's auc: 0.773593	valid_1's binary_logloss: 0.241462
[400]	training's auc: 0.829354	training's binary_logloss: 0.222879	valid_1's auc: 0.776099	valid_1's binary_logloss: 0.240594
[500]	training's auc: 0.841123	training's binary_logloss: 0.218383	valid_1's auc: 0.777284	valid_1's binary_logloss: 0.24014
|  28       |  0.7773   |  0.7425   |  451.4    |  15.55    |  126.2    |  24.37    |  62.57    |  22.14    |  6.783    |  0.7467   |


/usr/local/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[100]	training's auc: 0.778241	training's binary_logloss: 0.243862	valid_1's auc: 0.760175	valid_1's binary_logloss: 0.24783
[200]	training's auc: 0.797457	training's binary_logloss: 0.235008	valid_1's auc: 0.769726	valid_1's binary_logloss: 0.242954
[300]	training's auc: 0.812191	training's binary_logloss: 0.229352	valid_1's auc: 0.774143	valid_1's binary_logloss: 0.241295
[400]	training's auc: 0.824886	training's binary_logloss: 0.224612	valid_1's auc: 0.776502	valid_1's binary_logloss: 0.240437
[500]	training's auc: 0.835659	training's binary_logloss: 0.220499	valid_1's auc: 0.777928	valid_1's binary_logloss: 0.239939
|  29       |  0.7779   |  0.5288   |  298.4    |  14.09    |  182.4    |  37.65    |  54.36    |  7.913    |  3.01     |  0.7732   |


/usr/local/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[100]	training's auc: 0.771096	training's binary_logloss: 0.245213	valid_1's auc: 0.757144	valid_1's binary_logloss: 0.248184
[200]	training's auc: 0.790119	training's binary_logloss: 0.237296	valid_1's auc: 0.767762	valid_1's binary_logloss: 0.243565
[300]	training's auc: 0.80154	training's binary_logloss: 0.23285	valid_1's auc: 0.771849	valid_1's binary_logloss: 0.242007
[400]	training's auc: 0.810928	training's binary_logloss: 0.229355	valid_1's auc: 0.774467	valid_1's binary_logloss: 0.241101
[500]	training's auc: 0.818607	training's binary_logloss: 0.226514	valid_1's auc: 0.775923	valid_1's binary_logloss: 0.240616
|  30       |  0.7759   |  0.7962   |  309.4    |  7.089    |  194.4    |  35.83    |  40.28    |  5.876    |  1.207    |  0.6585   |


In [19]:
target_list = [i['target'] for i in lgb_BO.res]
print(target_list)
np.argmax(target_list)


[0.7767775166083138, 0.7753234902998535, 0.7773659822337948, 0.7762755864013193, 0.7764662874121195, 0.7769144931629292, 0.7773323675172852, 0.7765998773976205, 0.7761438056570105, 0.7767908849787016, 0.7772510072592936, 0.7771381338002419, 0.7765177975731143, 0.7765200944675512, 0.7771203622546268, 0.7769612332174275, 0.7759039739788991, 0.7757087252441897, 0.7758943321052947, 0.7754516922729547, 0.7760323760963289, 0.7759695323656197, 0.7758817500161976, 0.7753360088510683, 0.7771318657881756, 0.7775506566764805, 0.7755556736054107, 0.7772844599542038, 0.7779281527067806, 0.7759228876179445]


28

In [23]:
max_dict = lgb_BO.res[np.argmax(target_list)]
max_dict

{'target': 0.7779281527067806,
 'params': {'colsample_bytree': 0.5288123810873695,
  'max_bin': 298.3632940551493,
  'max_depth': 14.085247061998437,
  'min_child_samples': 182.4305882973765,
  'min_child_weight': 37.647486427741015,
  'num_leaves': 54.35658240763554,
  'reg_alpha': 7.912768101857651,
  'reg_lambda': 3.0095185838236933,
  'subsample': 0.773150758899048}}

In [24]:
def train_apps_all(apps_all_train):
    ftr_app = apps_all_train.drop(['SK_ID_CURR', 'TARGET'], axis=1)
    target_app = apps_all_train['TARGET']

    train_x, valid_x, train_y, valid_y = train_test_split(ftr_app, target_app, test_size=0.3, random_state=2020)
    print('train shape:', train_x.shape, 'valid shape:', valid_x.shape)
    clf = LGBMClassifier(
                nthread=4,
                n_estimators=2000,
                learning_rate=0.01,
                num_leaves=54,
                colsample_bytree=0.528,
                subsample=0.773,
                max_depth=14,
                reg_alpha=7.9,
                reg_lambda=3.0,
                min_child_samples=182,
                min_child_weight=40,
                max_bin=298,
                silent=-1,
                verbose=-1,
                )

    clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], eval_metric= 'auc', verbose= 100, 
                early_stopping_rounds= 100)
    
    return clf

In [25]:
clf = train_apps_all(apps_all_train)

train shape: (215257, 174) valid shape: (92254, 174)


/usr/local/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:598: UserWarning: 'silent' argument is deprecated and will be removed in a future release of LightGBM. Pass 'verbose' parameter via keyword arguments instead.
  _log_warning("'silent' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will

[LightGBM] [Warning] num_threads is set with nthread=4, will be overridden by n_jobs=-1. Current value: num_threads=-1
[100]	training's auc: 0.766438	training's binary_logloss: 0.253402	valid_1's auc: 0.75313	valid_1's binary_logloss: 0.255042
[200]	training's auc: 0.776961	training's binary_logloss: 0.244537	valid_1's auc: 0.759403	valid_1's binary_logloss: 0.248198
[300]	training's auc: 0.786671	training's binary_logloss: 0.239589	valid_1's auc: 0.765017	valid_1's binary_logloss: 0.245041
[400]	training's auc: 0.794819	training's binary_logloss: 0.236041	valid_1's auc: 0.769007	valid_1's binary_logloss: 0.24325
[500]	training's auc: 0.802199	training's binary_logloss: 0.233146	valid_1's auc: 0.771851	valid_1's binary_logloss: 0.242157
[600]	training's auc: 0.808497	training's binary_logloss: 0.230725	valid_1's auc: 0.77368	valid_1's binary_logloss: 0.241471
[700]	training's auc: 0.814395	training's binary_logloss: 0.228511	valid_1's auc: 0.775078	valid_1's binary_logloss: 0.240963
[8

In [26]:
preds = clf.predict_proba(apps_all_test.drop('SK_ID_CURR', axis=1))[:, 1]
apps_all_test['TARGET'] = preds
apps_all_test[['SK_ID_CURR', 'TARGET']].to_csv('prev_baseline_tuning_01.csv', index=False)